In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

from matplotlib_inline import backend_inline
backend_inline.set_matplotlib_formats('retina')

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import os
import warnings

import keras
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.utils import image_dataset_from_directory
import tensorflow as tf

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [3]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [4]:
import time
import multiprocessing
import pandas as pd

In [5]:
from tensorflow.python.keras.backend import set_session
from tensorflow.python.keras.backend import clear_session
from tensorflow.python.keras.backend import get_session
import tensorflow
import gc

# Reset Keras Session
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del model # this is from global space - change this as you need
    except:
        pass

    print(gc.collect()) # if it's done something you should see a number being outputted

    # use the same config as you used to create the session
    config = tensorflow.compat.v1.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    set_session(tensorflow.compat.v1.Session(config=config))

In [6]:
train_ds, validation_ds = image_dataset_from_directory(
    directory='weather_dataset/',
    labels='inferred',
    label_mode='categorical',
    batch_size=32, 
    validation_split = 0.3,
    subset = 'both',
    seed = 8675309)

Found 6862 files belonging to 11 classes.
Using 4804 files for training.
Using 2058 files for validation.


In [7]:
validation_ds, test_ds = tf.keras.utils.split_dataset(validation_ds, left_size=0.5, shuffle=True, seed=2)

### Perform a grid search to tune hyperparameters

In [8]:
def compile_model(optimizer=keras.optimizers.Adam, learning_rate=1e-3, layer1size=1024, trainable_layers=2):
    
    # create the base pre-trained model
    base_model = VGG16(weights='imagenet', include_top=False)

    x = base_model.output
    x1 = GlobalAveragePooling2D()(x)
    x2 = Dense(layer1size, activation='relu')(x1)
    predictions = Dense(11, activation='softmax')(x2)

    model = Model(inputs=base_model.input, outputs=predictions)

    # we chose to train the top 2 layers
    for layer in model.layers[:-trainable_layers]:
        layer.trainable = False
    for layer in model.layers[-trainable_layers:]:
        layer.trainable = True

    # compile the model (should be done *after* setting layers to non-trainable)
    model.compile(optimizer=optimizer(learning_rate=learning_rate), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    return model

In [9]:
k = 4
epochs = 10
batch_size = 32

In [10]:
n = train_ds.cardinality().numpy()
dataset_folds = []
for j in range(k):
    start = j * n // k
    end = (j+1) * n // k
    val_dataset = train_ds.take(end-start)
    train_dataset = train_ds.skip(end-start)
    dataset_folds.append((train_dataset, val_dataset))
print('data folded')

data folded


In [36]:
optimizers = [keras.optimizers.Adam, keras.optimizers.SGD, keras.optimizers.RMSprop]
learn_rates = [1e-4, 1e-3] #np.logspace(-4, -3, 2)
layer1sizes=[4096, 1024]
trainable_layers=[1, 2, 3]

In [32]:
counter = 1

#CVscores = []

for optimizer in optimizers:
    for learn_rate in learn_rates:
        for layer1size in layer1sizes:
            for trainable_layer in trainable_layers:
                print(counter, 'of', len(optimizers)*len(learn_rates)*len(layer1sizes)*len(trainable_layers))
                print(optimizer, learn_rate, layer1size, trainable_layer)

                scores = []
                for i in range(k): 
                    print('CV', i+1, '/', k)

                    model = compile_model(optimizer=optimizer, learning_rate=learn_rate, trainable_layers=trainable_layer, layer1size=layer1size)

                    model.fit(dataset_folds[i][0], 
                              epochs=epochs,
                              batch_size=batch_size, 
                              verbose=1)
                    scores.append(model.evaluate(dataset_folds[i][1])[1])

                print('score: ', np.mean(scores))
                CVscores.append(np.mean(scores))
                counter += 1
                print('GPU mem used :', tf.config.experimental.get_memory_info('GPU:0')['current']/1e9)
                if tf.config.experimental.get_memory_info('GPU:0')['current']/1e9 >= 10:
                    reset_keras()

1 of 24
<class 'keras.optimizers.optimizer_v2.gradient_descent.SGD'> 0.0001 4096 1
CV 1 / 4
Epoch 1/10
114/114 [==============================] - 7s 50ms/step - loss: 2.6822 - accuracy: 0.1898
Epoch 2/10
114/114 [==============================] - 6s 50ms/step - loss: 2.1845 - accuracy: 0.2856
Epoch 3/10
114/114 [==============================] - 6s 49ms/step - loss: 1.9254 - accuracy: 0.3528
Epoch 4/10
114/114 [==============================] - 6s 50ms/step - loss: 1.7432 - accuracy: 0.4075
Epoch 5/10
114/114 [==============================] - 6s 50ms/step - loss: 1.5764 - accuracy: 0.4715
Epoch 6/10
114/114 [==============================] - 6s 50ms/step - loss: 1.4595 - accuracy: 0.5130
Epoch 7/10
114/114 [==============================] - 6s 50ms/step - loss: 1.3738 - accuracy: 0.5533
Epoch 8/10
114/114 [==============================] - 6s 50ms/step - loss: 1.2938 - accuracy: 0.5820
Epoch 9/10
114/114 [==============================] - 6s 50ms/step - loss: 1.2262 - accuracy: 0.6094

113/113 [==============================] - 6s 50ms/step - loss: 1.4135 - accuracy: 0.5468
Epoch 4/10
113/113 [==============================] - 6s 50ms/step - loss: 1.2191 - accuracy: 0.6179
Epoch 5/10
113/113 [==============================] - 6s 50ms/step - loss: 1.0962 - accuracy: 0.6572
Epoch 6/10
113/113 [==============================] - 6s 50ms/step - loss: 1.0073 - accuracy: 0.6839
Epoch 7/10
113/113 [==============================] - 6s 50ms/step - loss: 0.9415 - accuracy: 0.7029
Epoch 8/10
113/113 [==============================] - 6s 50ms/step - loss: 0.8897 - accuracy: 0.7182
Epoch 9/10
113/113 [==============================] - 6s 50ms/step - loss: 0.8422 - accuracy: 0.7327
Epoch 10/10
38/38 [==============================] - 2s 50ms/step - loss: 0.8517 - accuracy: 0.7319
score:  0.7365920692682266
GPU mem used : 1.910610176
3 of 24
<class 'keras.optimizers.optimizer_v2.gradient_descent.SGD'> 0.0001 4096 3
CV 1 / 4
Epoch 1/10
114/114 [==============================] - 7s 5

113/113 [==============================] - 6s 50ms/step - loss: 1.6102 - accuracy: 0.4955
Epoch 6/10
113/113 [==============================] - 6s 50ms/step - loss: 1.4491 - accuracy: 0.5407
Epoch 7/10
113/113 [==============================] - 6s 51ms/step - loss: 1.3281 - accuracy: 0.5792
Epoch 8/10
113/113 [==============================] - 6s 50ms/step - loss: 1.2294 - accuracy: 0.6065
Epoch 9/10
113/113 [==============================] - 6s 51ms/step - loss: 1.1450 - accuracy: 0.6307
Epoch 10/10
38/38 [==============================] - 2s 50ms/step - loss: 1.1404 - accuracy: 0.6365
CV 3 / 4
Epoch 1/10
113/113 [==============================] - 7s 50ms/step - loss: 3.3095 - accuracy: 0.1644
Epoch 2/10
113/113 [==============================] - 6s 50ms/step - loss: 2.4826 - accuracy: 0.2759
Epoch 3/10
113/113 [==============================] - 6s 50ms/step - loss: 2.0170 - accuracy: 0.3682
Epoch 4/10
113/113 [==============================] - 6s 50ms/step - loss: 1.7120 - accuracy: 

113/113 [==============================] - 6s 51ms/step - loss: 0.3417 - accuracy: 0.9013
Epoch 10/10
38/38 [==============================] - 2s 51ms/step - loss: 0.5009 - accuracy: 0.8224
score:  0.8332370072603226
GPU mem used : 1.835608064
10 of 24
<class 'keras.optimizers.optimizer_v2.gradient_descent.SGD'> 0.001 1024 1
CV 1 / 4
Epoch 1/10
114/114 [==============================] - 7s 51ms/step - loss: 2.6184 - accuracy: 0.2845
Epoch 2/10
114/114 [==============================] - 6s 50ms/step - loss: 1.4719 - accuracy: 0.5238
Epoch 3/10
114/114 [==============================] - 6s 50ms/step - loss: 1.1483 - accuracy: 0.6254
Epoch 4/10
114/114 [==============================] - 6s 50ms/step - loss: 0.9903 - accuracy: 0.6751
Epoch 5/10
114/114 [==============================] - 6s 50ms/step - loss: 0.8796 - accuracy: 0.7019
Epoch 6/10
114/114 [==============================] - 6s 50ms/step - loss: 0.8138 - accuracy: 0.7301
Epoch 7/10
114/114 [==============================] - 6s 5

38/38 [==============================] - 2s 50ms/step - loss: 0.4672 - accuracy: 0.8487
CV 3 / 4
Epoch 1/10
113/113 [==============================] - 7s 50ms/step - loss: 1.5145 - accuracy: 0.5106
Epoch 2/10
113/113 [==============================] - 6s 50ms/step - loss: 0.7825 - accuracy: 0.7598
Epoch 3/10
113/113 [==============================] - 6s 50ms/step - loss: 0.6103 - accuracy: 0.8049
Epoch 4/10
113/113 [==============================] - 6s 50ms/step - loss: 0.5239 - accuracy: 0.8344
Epoch 5/10
113/113 [==============================] - 6s 50ms/step - loss: 0.4627 - accuracy: 0.8540
Epoch 6/10
113/113 [==============================] - 6s 50ms/step - loss: 0.4188 - accuracy: 0.8662
Epoch 7/10
113/113 [==============================] - 6s 51ms/step - loss: 0.3824 - accuracy: 0.8802
Epoch 8/10
113/113 [==============================] - 6s 50ms/step - loss: 0.3569 - accuracy: 0.8888
Epoch 9/10
113/113 [==============================] - 6s 50ms/step - loss: 0.3324 - accuracy: 0

114/114 [==============================] - 6s 51ms/step - loss: 0.5574 - accuracy: 0.8182
Epoch 3/10
114/114 [==============================] - 6s 50ms/step - loss: 0.3792 - accuracy: 0.8807
Epoch 4/10
114/114 [==============================] - 6s 51ms/step - loss: 0.2887 - accuracy: 0.9113
Epoch 5/10
114/114 [==============================] - 6s 51ms/step - loss: 0.2201 - accuracy: 0.9351
Epoch 6/10
114/114 [==============================] - 6s 51ms/step - loss: 0.1720 - accuracy: 0.9517
Epoch 7/10
114/114 [==============================] - 6s 50ms/step - loss: 0.1356 - accuracy: 0.9622
Epoch 8/10
114/114 [==============================] - 6s 51ms/step - loss: 0.1117 - accuracy: 0.9754
Epoch 9/10
114/114 [==============================] - 6s 50ms/step - loss: 0.0903 - accuracy: 0.9801
Epoch 10/10
37/37 [==============================] - 2s 50ms/step - loss: 0.4480 - accuracy: 0.8708
CV 2 / 4
Epoch 1/10
113/113 [==============================] - 7s 51ms/step - loss: 1.2710 - accuracy: 

113/113 [==============================] - 6s 52ms/step - loss: 0.2249 - accuracy: 0.9423
Epoch 7/10
113/113 [==============================] - 7s 51ms/step - loss: 0.2119 - accuracy: 0.9462
Epoch 8/10
113/113 [==============================] - 6s 50ms/step - loss: 0.1980 - accuracy: 0.9546
Epoch 9/10
113/113 [==============================] - 6s 51ms/step - loss: 0.1500 - accuracy: 0.9657
Epoch 10/10
38/38 [==============================] - 2s 52ms/step - loss: 1.5686 - accuracy: 0.8199
CV 4 / 4
Epoch 1/10
113/113 [==============================] - 8s 51ms/step - loss: 2.1977 - accuracy: 0.7051
Epoch 2/10
113/113 [==============================] - 6s 51ms/step - loss: 0.5544 - accuracy: 0.8344
Epoch 3/10
113/113 [==============================] - 6s 50ms/step - loss: 0.4156 - accuracy: 0.8760
Epoch 4/10
113/113 [==============================] - 6s 51ms/step - loss: 0.3171 - accuracy: 0.9114
Epoch 5/10
113/113 [==============================] - 6s 51ms/step - loss: 0.2430 - accuracy: 

114/114 [==============================] - 6s 51ms/step - loss: 0.1092 - accuracy: 0.9696
Epoch 9/10
114/114 [==============================] - 6s 51ms/step - loss: 0.0921 - accuracy: 0.9751
Epoch 10/10
37/37 [==============================] - 3s 51ms/step - loss: 0.7284 - accuracy: 0.8860
CV 2 / 4
Epoch 1/10
113/113 [==============================] - 7s 51ms/step - loss: 1.3827 - accuracy: 0.7152
Epoch 2/10
113/113 [==============================] - 6s 51ms/step - loss: 0.4689 - accuracy: 0.8576
Epoch 3/10
113/113 [==============================] - 6s 51ms/step - loss: 0.2918 - accuracy: 0.9066
Epoch 4/10
113/113 [==============================] - 6s 51ms/step - loss: 0.2124 - accuracy: 0.9303
Epoch 5/10
113/113 [==============================] - 6s 51ms/step - loss: 0.1719 - accuracy: 0.9476
Epoch 6/10
113/113 [==============================] - 6s 51ms/step - loss: 0.1331 - accuracy: 0.9643
Epoch 7/10
113/113 [==============================] - 6s 51ms/step - loss: 0.1242 - accuracy: 

In [35]:
CVscores

[0.8406550139188766,
 0.7601795792579651,
 0.8723384141921997,
 0.8483674824237823,
 0.6226106882095337,
 0.42706481367349625,
 0.815817266702652,
 0.7548008561134338,
 0.8393214344978333,
 0.7746599465608597,
 0.8184677213430405,
 0.8206570148468018,
 0.618865579366684,
 0.7365920692682266,
 0.7348584085702896,
 0.45060233026742935,
 0.6388135701417923,
 0.6456870287656784,
 0.8199791014194489,
 0.8410495072603226,
 0.8332370072603226,
 0.7621577233076096,
 0.8170563578605652,
 0.8123888820409775,
 0.844177857041359,
 0.8779005259275436,
 0.8830514401197433,
 0.7793552279472351,
 0.8749722093343735,
 0.866715207695961,
 0.8437055349349976,
 0.8331703245639801,
 0.8652871549129486,
 0.8369432389736176,
 0.8371432721614838,
 0.8820623457431793]

In [69]:
results = []
i = 0
for optimizer in optimizers:
    for learn_rate in learn_rates:
        for layer1size in layer1sizes:
            for trainable_layer in trainable_layers:
                results.append([CVscores[i], optimizer, learn_rate, layer1size, trainable_layer])
                i += 1

results = pd.DataFrame(results, columns=['score', 'optimizer', 'learning_rate', 'layer1size', 'tunable_layers'])

In [71]:
results.sort_values('score', ascending=False)

,score,optimizer,learning_rate,layer1size,tunable_layers
26,0.883051,<class 'keras.optimizers.optimizer_v2.rmsprop....,0.0001,4096,3
35,0.882062,<class 'keras.optimizers.optimizer_v2.rmsprop....,0.0010,1024,3
25,0.877901,<class 'keras.optimizers.optimizer_v2.rmsprop....,0.0001,4096,2
28,0.874972,<class 'keras.optimizers.optimizer_v2.rmsprop....,0.0001,1024,2
2,0.872338,<class 'keras.optimizers.optimizer_v2.adam.Adam'>,0.0001,4096,3
29,0.866715,<class 'keras.optimizers.optimizer_v2.rmsprop....,0.0001,1024,3
32,0.865287,<class 'keras.optimizers.optimizer_v2.rmsprop....,0.0010,4096,3
3,0.848367,<class 'keras.optimizers.optimizer_v2.adam.Adam'>,0.0001,1024,1
24,0.844178,<class 'keras.optimizers.optimizer_v2.rmsprop....,0.0001,4096,1
30,0.843706,<class 'keras.optimizers.optimizer_v2.rmsprop....,0.0010,4096,1


Best results (though it's close) were with 
- RMSprop
- 4096 layer size
- Top 3 tunable layers
- 0.0001 learning rate

### Using those hyperparameters, tune it for many epochs

In [75]:
bigtrain_epochs = 50

bigtrain_model = compile_model(optimizer=keras.optimizers.RMSprop, learning_rate=1e-4, trainable_layers=3, layer1size=4096)

bigtrain_model.fit(train_ds, 
                   epochs=bigtrain_epochs,
                   batch_size=batch_size, 
                   verbose=1, 
                   validation_data=validation_ds)

Epoch 1/50
151/151 [==============================] - 12s 74ms/step - loss: 0.7874 - accuracy: 0.7519 - val_loss: 0.4946 - val_accuracy: 0.8338
Epoch 2/50
151/151 [==============================] - 10s 62ms/step - loss: 0.3580 - accuracy: 0.8832 - val_loss: 0.4566 - val_accuracy: 0.8426
Epoch 3/50
151/151 [==============================] - 10s 63ms/step - loss: 0.2364 - accuracy: 0.9215 - val_loss: 0.5695 - val_accuracy: 0.8173
Epoch 4/50
151/151 [==============================] - 10s 63ms/step - loss: 0.1697 - accuracy: 0.9438 - val_loss: 0.5242 - val_accuracy: 0.8533
Epoch 5/50
151/151 [==============================] - 10s 63ms/step - loss: 0.1185 - accuracy: 0.9634 - val_loss: 0.7008 - val_accuracy: 0.8056
Epoch 6/50
151/151 [==============================] - 10s 62ms/step - loss: 0.0815 - accuracy: 0.9773 - val_loss: 0.6101 - val_accuracy: 0.8338
Epoch 7/50
151/151 [==============================] - 10s 64ms/step - loss: 0.0603 - accuracy: 0.9840 - val_loss: 0.4879 - val_accuracy:

KeyboardInterrupt: 

In [11]:
train_ds_singlebatch, validation_ds_singlebatch = image_dataset_from_directory(
    directory='weather_dataset/',
    labels='inferred',
    label_mode='categorical',
    batch_size=1, 
    validation_split = 0.3,
    subset = 'both',
    seed = 8675309)

validation_ds_singlebatch, test_ds_singlebatch = tf.keras.utils.split_dataset(train_ds_singlebatch, left_size=0.5, shuffle=True, seed=2)

Found 6862 files belonging to 11 classes.
Using 4804 files for training.
Using 2058 files for validation.


In [12]:
bigtrain_epochs = 50

bigtrain_model = compile_model(optimizer=keras.optimizers.RMSprop, learning_rate=1e-4, trainable_layers=3, layer1size=4096)

bigtrain_model.fit(train_ds_singlebatch, 
                   epochs=bigtrain_epochs,
                   batch_size=batch_size, 
                   verbose=1, 
                   validation_data=validation_ds_singlebatch)

Epoch 1/50
4804/4804 [==============================] - 32s 6ms/step - loss: 0.9965 - accuracy: 0.7692 - val_loss: 0.5698 - val_accuracy: 0.8655
Epoch 2/50
4804/4804 [==============================] - 26s 5ms/step - loss: 0.6456 - accuracy: 0.8535 - val_loss: 0.4977 - val_accuracy: 0.8913
Epoch 3/50
4804/4804 [==============================] - 26s 5ms/step - loss: 0.5335 - accuracy: 0.8855 - val_loss: 0.3715 - val_accuracy: 0.9221
Epoch 4/50
4804/4804 [==============================] - 26s 5ms/step - loss: 0.4881 - accuracy: 0.9045 - val_loss: 0.3247 - val_accuracy: 0.9334
Epoch 5/50
4804/4804 [==============================] - 26s 5ms/step - loss: 0.4294 - accuracy: 0.9184 - val_loss: 0.3751 - val_accuracy: 0.9267
Epoch 6/50
4804/4804 [==============================] - 26s 5ms/step - loss: 0.3970 - accuracy: 0.9269 - val_loss: 0.2480 - val_accuracy: 0.9534
Epoch 7/50
4804/4804 [==============================] - 26s 5ms/step - loss: 0.3715 - accuracy: 0.9396 - val_loss: 0.2956 - val_ac

In [13]:
bigtrain_model.evaluate(test_ds_singlebatch)

2402/2402 [==============================] - 9s 3ms/step - loss: 0.2253 - accuracy: 0.9742


[0.22533348202705383, 0.9741881489753723]

Some stats about model performance:
- Training accuracy: 98.73%
- Validation accuracy: 96.88%
- Test accuracy: 97.42%

I'm pretty happy with this. 

Below I try a small number of images I downloaded from the internet and see if these categories are accurate as well.

In [16]:
def pred_to_cat(prediction, probabilities=False):
    if not probabilities:
        print(np.array(train_ds_singlebatch.class_names)[np.argmax(prediction)])
    else:
        for i in range(len(prediction[0])):
            print('%.4f'%np.sort(prediction)[:,::-1][0][i], \
                  np.array(train_ds_singlebatch.class_names)[np.where(np.sort(prediction)[:,::-1][0][i] == prediction[0])[0][0]])

In [19]:
fog1 = tf.keras.utils.load_img(
    'weather_manual_test/fog1.jpg',
    grayscale=False,
    color_mode="rgb",
    target_size=(256, 256),
    interpolation="nearest",
    keep_aspect_ratio=False,
)

fog1 = np.asarray(fog1)

fog1 = np.reshape(fog1, [1,256,256,3])

pred_to_cat(bigtrain_model.predict(fog1, verbose = 0), probabilities=True)

1.0000 sandstorm
0.0000 rime
0.0000 lightning
0.0000 snow
0.0000 rain
0.0000 rainbow
0.0000 fogsmog
0.0000 hail
0.0000 frost
0.0000 dew
0.0000 glaze


In [29]:
fog2 = tf.keras.utils.load_img(
    'weather_manual_test/fog2.jpeg',
    grayscale=False,
    color_mode="rgb",
    target_size=(256, 256),
    interpolation="nearest",
    keep_aspect_ratio=False,
)

fog2 = np.asarray(fog2)

fog2 = np.reshape(fog2, [1,256,256,3])

pred_to_cat(bigtrain_model.predict(fog2, verbose = 0), probabilities=True)

1.0000 fogsmog
0.0000 lightning
0.0000 rain
0.0000 rainbow
0.0000 dew
0.0000 dew
0.0000 dew
0.0000 dew
0.0000 dew
0.0000 dew
0.0000 dew


In [20]:
rain1 = tf.keras.utils.load_img(
    'weather_manual_test/rain1.jpeg',
    grayscale=False,
    color_mode="rgb",
    target_size=(256, 256),
    interpolation="nearest",
    keep_aspect_ratio=False,
)

rain1 = np.asarray(rain1)

rain1 = np.reshape(rain1, [1,256,256,3])

pred_to_cat(bigtrain_model.predict(rain1, verbose = 0), probabilities=True)

1.0000 rain
0.0000 fogsmog
0.0000 hail
0.0000 snow
0.0000 frost
0.0000 dew
0.0000 rainbow
0.0000 rime
0.0000 sandstorm
0.0000 glaze
0.0000 lightning


In [21]:
rain2 = tf.keras.utils.load_img(
    'weather_manual_test/rain2.jpeg',
    grayscale=False,
    color_mode="rgb",
    target_size=(256, 256),
    interpolation="nearest",
    keep_aspect_ratio=False,
)

rain2 = np.asarray(rain2)

rain2 = np.reshape(rain2, [1,256,256,3])

pred_to_cat(bigtrain_model.predict(rain2, verbose = 0), probabilities=True)

1.0000 rain
0.0000 frost
0.0000 snow
0.0000 dew
0.0000 rainbow
0.0000 hail
0.0000 fogsmog
0.0000 fogsmog
0.0000 fogsmog
0.0000 fogsmog
0.0000 fogsmog


In [22]:
rain3 = tf.keras.utils.load_img(
    'weather_dataset/rain/1171.jpg',
    grayscale=False,
    color_mode="rgb",
    target_size=(256, 256),
    interpolation="nearest",
    keep_aspect_ratio=False,
)

rain3 = np.asarray(rain3)

rain3 = np.reshape(rain3, [1,256,256,3])

pred_to_cat(bigtrain_model.predict(rain3, verbose = 0), probabilities=True)

1.0000 rain
0.0000 dew
0.0000 dew
0.0000 dew
0.0000 dew
0.0000 dew
0.0000 dew
0.0000 dew
0.0000 dew
0.0000 dew
0.0000 dew


In [24]:
rime1 = tf.keras.utils.load_img(
    'weather_manual_test/rime1.jpeg',
    grayscale=False,
    color_mode="rgb",
    target_size=(256, 256),
    interpolation="nearest",
    keep_aspect_ratio=False,
)

rime1 = np.asarray(rime1)

rime1 = np.reshape(rime1, [1,256,256,3])

pred_to_cat(bigtrain_model.predict(rime1, verbose = 0), probabilities=True)

1.0000 rime
0.0000 glaze
0.0000 dew
0.0000 dew
0.0000 dew
0.0000 dew
0.0000 dew
0.0000 dew
0.0000 dew
0.0000 dew
0.0000 dew


In [30]:
rime2 = tf.keras.utils.load_img(
    'weather_manual_test/rime2.jpeg',
    grayscale=False,
    color_mode="rgb",
    target_size=(256, 256),
    interpolation="nearest",
    keep_aspect_ratio=False,
)

rime2 = np.asarray(rime2)

rime2 = np.reshape(rime2, [1,256,256,3])

pred_to_cat(bigtrain_model.predict(rime2, verbose = 0), probabilities=True)

1.0000 rime
0.0000 snow
0.0000 frost
0.0000 glaze
0.0000 sandstorm
0.0000 fogsmog
0.0000 lightning
0.0000 hail
0.0000 rain
0.0000 dew
0.0000 rainbow


In [32]:
sandstorm1 = tf.keras.utils.load_img(
    'weather_manual_test/sandstorm1.jpeg',
    grayscale=False,
    color_mode="rgb",
    target_size=(256, 256),
    interpolation="nearest",
    keep_aspect_ratio=False,
)

sandstorm1 = np.asarray(sandstorm1)

sandstorm1 = np.reshape(sandstorm1, [1,256,256,3])

pred_to_cat(bigtrain_model.predict(sandstorm1, verbose = 0), probabilities=True)

1.0000 sandstorm
0.0000 snow
0.0000 rainbow
0.0000 rain
0.0000 lightning
0.0000 glaze
0.0000 dew
0.0000 dew
0.0000 dew
0.0000 dew
0.0000 dew


In [35]:
snow1 = tf.keras.utils.load_img(
    'weather_manual_test/snow1.jpeg',
    grayscale=False,
    color_mode="rgb",
    target_size=(256, 256),
    interpolation="nearest",
    keep_aspect_ratio=False,
)

snow1 = np.asarray(snow1)

snow1 = np.reshape(snow1, [1,256,256,3])

pred_to_cat(bigtrain_model.predict(snow1, verbose = 0), probabilities=True)

0.9966 rime
0.0034 rain
0.0000 snow
0.0000 glaze
0.0000 frost
0.0000 hail
0.0000 dew
0.0000 dew
0.0000 dew
0.0000 dew
0.0000 dew


In [36]:
glaze1 = tf.keras.utils.load_img(
    'weather_manual_test/glaze1.jpeg',
    grayscale=False,
    color_mode="rgb",
    target_size=(256, 256),
    interpolation="nearest",
    keep_aspect_ratio=False,
)

glaze1 = np.asarray(glaze1)

glaze1 = np.reshape(glaze1, [1,256,256,3])

pred_to_cat(bigtrain_model.predict(glaze1, verbose = 0), probabilities=True)

1.0000 frost
0.0000 rime
0.0000 glaze
0.0000 rain
0.0000 hail
0.0000 dew
0.0000 lightning
0.0000 fogsmog
0.0000 rainbow
0.0000 snow
0.0000 sandstorm


Of the 10 manual images I've tried, it's gotten 7 of them correct. This is pretty bad considering it did 97% on the test dataset. I'm left to conclude one or more of the following:
1) I messed up the data partitioning and gave it some test data without realizing.   
2) This dataset is not representative and probably too small. 